In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
folder_path = 'Data'

In [3]:
# Initialize empty dictionaries for 'unsold' and 'supply' DataFrames
unsold_dataframes = {}
supply_dataframes = {}

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # Extract month and year from the filename
        month_year = filename.split(' ')[-1].split('.')[0]  # Extracting the month'year from the filename

        # Read the Excel file into a dictionary of DataFrames (one DataFrame per sheet)
        excel_data = pd.read_excel(file_path, sheet_name=None, header=1)

        # Iterate through sheets and store each DataFrame in the respective dictionaries
        for sheet_name, df in excel_data.items():
            df_name = f"{sheet_name}_{month_year}"
            
            if sheet_name.lower() == 'unsold':
                unsold_dataframes[df_name] = df
            elif sheet_name.lower() == 'supply':
                supply_dataframes[df_name] = df

In [4]:
def convert_supply_dataframe(dataframe, month_year_str, column_name):
    df_new = dataframe.drop(['Avg. PO', 'Total', 'Week', 'Sun', 'Avg. Wk Supply', 'Avg. Sun Supply'], axis=1)
    df_new = pd.melt(df_new, id_vars = ['CODE', 'AGENT NAME', 'STATION'], var_name='Date', value_name=column_name)
    df_new[['Day', 'Day_of_week']] = df_new['Date'].str.split(' ', 1, expand=True)
    df_new['Day'] = pd.to_numeric(df_new['Day'], errors='coerce')
    # Extract month and year from the input string
    month_year = month_year_str.strip().replace("'", " ").split(" ")
    month, year = month_year[0], month_year[1]
    # Create a new 'Date' column with the format 'YYYY-MM-DD'
    month_str = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'June': '06',
                 'July': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}[month]
    # Create the datetime col
    df_new['Date'] = pd.to_datetime(f'20{year}-{month_str}-' + df_new['Day'].astype(str), errors='coerce')
    df_new = df_new.dropna(subset=['CODE'])
    df_new.loc[:,'AGENT NAME'] = df_new['AGENT NAME'].fillna(method='ffill').copy()
    df_new = df_new.drop(['Day', 'Day_of_week'], axis=1)
    return df_new

In [5]:
def convert_unsold_dataframe(dataframe, column_name):
    df_new = dataframe.drop(['Total', 'Wkday', 'Sunday'], axis=1)
    df_new = pd.melt(df_new, id_vars = ['CODE', 'AGENT NAME', 'STATION'], var_name='Date', value_name=column_name)
    df_new['Date'] = pd.to_datetime(df_new['Date'], format='%d.%m.%y', errors='coerce')
    df_new = df_new.dropna(subset=['CODE'])
    df_new.loc[:,'AGENT NAME'] = df_new['AGENT NAME'].fillna(method='ffill').copy()
    return df_new

In [6]:
converted_supply_dataframes = []  # List to store the converted DataFrames

# Example: Applying the function to a DataFrame in supply_dataframes
for key, dataframe in supply_dataframes.items():
    sheet_name, month_year_str = key.split('_')
    converted_df_name = f"{sheet_name}_converted_{month_year_str}"
    # Applying the conversion function
    converted_df = convert_supply_dataframe(dataframe, month_year_str, 'Supply')
    
    # Store the converted DataFrame in the list
    converted_supply_dataframes.append(converted_df)

# Concatenate all the converted DataFrames into one DataFrame
final_supply_dataframe = pd.concat(converted_supply_dataframes, ignore_index=True)

In [7]:
converted_unsold_dataframes = []  # List to store the converted DataFrames

# Example: Applying the function to a DataFrame in supply_dataframes
for key, dataframe in unsold_dataframes.items():
    sheet_name, month_year_str = key.split('_')
    converted_df_name = f"{sheet_name}_converted_{month_year_str}"
    # Applying the conversion function
    converted_df = convert_unsold_dataframe(dataframe, 'Unsold')
    
    # Store the converted DataFrame in the list
    converted_unsold_dataframes.append(converted_df)

# Concatenate all the converted DataFrames into one DataFrame
final_unsold_dataframe = pd.concat(converted_unsold_dataframes, ignore_index=True)

In [ ]:
final_supply_dataframe

In [ ]:
final_df = pd.merge(final_supply_dataframe, final_unsold_dataframe[['CODE', 'Date', 'Unsold']], on=['CODE', 'Date'], how='left')
final_df['Net_Sales'] = (final_df['Supply'] - final_df['Unsold'].fillna(0))
final_df

In [ ]:
final_df = final_df.sort_values(by='Date')
final_df['Day_of_the_Week'] = final_df['Date'].dt.day_name()
final_df

In [24]:
final_df.to_csv('Results/2023_sales_raw.csv')

In [ ]:
datewise_sales = final_df.groupby('Date').agg({
    'Supply': 'sum',
    'Unsold': 'sum',  # Assuming 'Unsold' is a numerical column, otherwise, modify the aggregation accordingly
    'Net_Sales': 'sum'
}).reset_index()
datewise_sales['Day_of_the_Week'] = datewise_sales['Date'].dt.day_name()

datewise_sales

In [19]:
datewise_sales.to_csv("Results/2023_Datewise_sales.csv")